In [0]:
from bs4 import BeautifulSoup
import requests as re
import smtplib
from email.mime.text import MIMEText
import os

# EMAIL SETTINGS
YOUR_EMAIL = "9009arun@gmail.com"
APP_PASSWORD = "emzr ounw yoom aoae"
SEND_TO = "9009arun@gmail.com"

def send_email(new_movies):
    body = "🔥 NEW Tamil Movies:\n\n"
    for m in new_movies:
        body += m + "\n"

    msg = MIMEText(body)
    msg["Subject"] = "TamilMV Updates"
    msg["From"] = YOUR_EMAIL
    msg["To"] = SEND_TO

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(YOUR_EMAIL, APP_PASSWORD)
        server.send_message(msg)


def scrape_movies():
    url = "https://www.1tamilmv.pink"
    response = re.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    blocks = soup.find_all("div", class_="ipsWidget_inner ipsPad ipsType_richText")
    movie_block = blocks[1]

    html = movie_block.decode_contents().replace("<br>", "\n")
    lines = html.split("\n")

    movies = []
    for line in lines:
        line_soup = BeautifulSoup(line, "html.parser")
        clean = line_soup.get_text().strip()

        if not clean or "(" not in clean:
            continue

        name = clean.split("(")[0].strip()
        year = clean.split("(")[1].split(")")[0].strip()
        after_year = clean.split(")")[1].strip()
        language = after_year.split()[0]

        if language.lower() != "tamil":
            continue

        a_tag = line_soup.find("a")
        link = a_tag.get("href") if a_tag else "No Link"

        movies.append(f"{name} ({year}) {after_year} : {link}")

    return movies


# -------- FILE STORAGE IN DATABRICKS --------
FILE_PATH = "/Volumes/workspace/practice_database/movies_checker_storage/movies_old.txt"

def load_old_list():
    if not os.path.exists(FILE_PATH):
        return []
    with open(FILE_PATH, "r", encoding="utf-8") as f:
        return [line.strip() for line in f.readlines()]

def save_list(movies):
    try:
        with open(FILE_PATH, "w", encoding="utf-8") as f:
            for m in movies:
                f.write(m + "\n")
    except Exception as e:
        print("Error writing file:", e)


# -------- MAIN --------
new_list = scrape_movies()
old_list = load_old_list()

new_updates = [m for m in new_list if m not in old_list]

if new_updates:
    print("New movies found — sending email.")
    send_email(new_updates)
else:
    print("No new updates.")

save_list(new_list)
